<a href="https://colab.research.google.com/github/AMEYABHAVE/Cristl/blob/main/project2_HAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.optimizers import Adam
print(tf.__version__)

2.6.0


In [2]:
from google.colab import drive
import pandas as pd
file_location = '/content/drive/MyDrive/accelerometer.csv'
accelerometer =pd.read_csv(file_location)
accelerometer.drop(columns=['time','repetition','uuid'], inplace=True)
accchair= accelerometer[accelerometer['activity'] == 'CHAIR'].head(31065)
acccouch= accelerometer[accelerometer['activity'] == 'COUCH'].head(31065)
accsurface= accelerometer[accelerometer['activity'] == 'SURFACE'].head(31065)
acclyingup= accelerometer[accelerometer['activity'] == 'LYING_UP'].head(31065)
acclyingside= accelerometer[accelerometer['activity'] == 'LYING_SIDE'].head(31065)
accdataset = pd.DataFrame()
accdataset = accdataset.append([accchair, acccouch, accsurface, acclyingup, acclyingside])

file_location1 = '/content/drive/MyDrive/gyroscope.csv'
gyroscope =pd.read_csv(file_location1)
gyroscope.drop(columns=['time','repetition','uuid'], inplace=True)
gyrchair= gyroscope[gyroscope['activity'] == 'CHAIR'].head(10178)
gyrcouch= gyroscope[gyroscope['activity'] == 'COUCH'].head(10178)
gyrsurface= gyroscope[gyroscope['activity'] == 'SURFACE'].head(10178)
gyrlyingup= gyroscope[gyroscope['activity'] == 'LYING_UP'].head(10178)
gyrlyingside= gyroscope[gyroscope['activity'] == 'LYING_SIDE'].head(10178)
gyrdataset = pd.DataFrame()
gyrdataset = gyrdataset.append([gyrchair, gyrcouch, gyrsurface, gyrlyingup, gyrlyingside])

In [3]:
accdataset.info()
gyrdataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 155325 entries, 0 to 290908
Data columns (total 5 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   name      155325 non-null  object 
 1   acc_x     155325 non-null  float64
 2   acc_y     155325 non-null  float64
 3   acc_z     155325 non-null  float64
 4   activity  155325 non-null  object 
dtypes: float64(3), object(2)
memory usage: 7.1+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 50890 entries, 0 to 93659
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   name      50890 non-null  object 
 1   gyr_x     50890 non-null  float64
 2   gyr_y     50890 non-null  float64
 3   gyr_z     50890 non-null  float64
 4   activity  50890 non-null  object 
dtypes: float64(3), object(2)
memory usage: 2.3+ MB


In [4]:
accdataset['activity'].value_counts()

SURFACE       31065
LYING_SIDE    31065
COUCH         31065
CHAIR         31065
LYING_UP      31065
Name: activity, dtype: int64

In [5]:
gyrdataset['activity'].value_counts()

COUCH         10178
SURFACE       10178
CHAIR         10178
LYING_UP      10178
LYING_SIDE    10178
Name: activity, dtype: int64

In [6]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
label=LabelEncoder()
accdataset['label']=label.fit_transform(accdataset['activity'])
print(accdataset.head())
print(label.classes_)

            name     acc_x     acc_y     acc_z activity  label
0  accelerometer -0.414825  7.590469  6.235336    CHAIR      0
1  accelerometer -0.337662  7.645508  6.125275    CHAIR      0
2  accelerometer -0.393295  7.605423  6.231155    CHAIR      0
3  accelerometer -0.469269  7.592865  6.281403    CHAIR      0
4  accelerometer -0.418411  7.633530  6.317307    CHAIR      0
['CHAIR' 'COUCH' 'LYING_SIDE' 'LYING_UP' 'SURFACE']


In [7]:
gyrdataset['label']=label.fit_transform(gyrdataset['activity'])
print(gyrdataset.head())
print(label.classes_)

        name     gyr_x     gyr_y     gyr_z activity  label
0  gyroscope -0.309540  7.679596  6.142029    CHAIR      0
1  gyroscope -0.530884  7.897354  5.765747    CHAIR      0
2  gyroscope -0.608643  8.231140  5.266846    CHAIR      0
3  gyroscope -0.570953  8.260452  5.229172    CHAIR      0
4  gyroscope -0.816223  8.244308  5.284195    CHAIR      0
['CHAIR' 'COUCH' 'LYING_SIDE' 'LYING_UP' 'SURFACE']


In [8]:
AX = accdataset[['acc_x','acc_y','acc_z']]
AY = accdataset['label']
scaler = StandardScaler()
AX = scaler.fit_transform(AX)
Ascaled_X = pd.DataFrame(data=AX, columns=['acc_x','acc_y','acc_z'])
Ascaled_X['label'] = AY.values
Ascaled_X

,acc_x,acc_y,acc_z,label
0,-0.274232,0.775611,0.338360,0
1,-0.251889,0.791256,0.317787,0
2,-0.267998,0.779862,0.337579,0
3,-0.289997,0.776292,0.346972,0
4,-0.275271,0.787851,0.353683,0
...,...,...,...,...
155320,1.850608,-0.116939,0.115499,2
155321,1.858230,-0.125440,0.105099,2
155322,1.861866,-0.124082,0.114717,2
155323,1.845412,-0.123401,0.119971,2


In [9]:
GX = gyrdataset[['gyr_x','gyr_y','gyr_z']]
GY = gyrdataset['label']
scaler = StandardScaler()
GX = scaler.fit_transform(GX)
Gscaled_X = pd.DataFrame(data=GX, columns=['gyr_x','gyr_y','gyr_z'])
Gscaled_X['label'] = GY.values
Gscaled_X

,gyr_x,gyr_y,gyr_z,label
0,-0.381119,0.954777,0.289859,0
1,-0.447922,1.017698,0.225016,0
2,-0.471390,1.114144,0.139043,0
3,-0.460015,1.122614,0.132551,0
4,-0.534039,1.117949,0.142032,0
...,...,...,...,...
50885,1.775278,-0.051905,0.156878,2
50886,1.748738,-0.061239,0.197183,2
50887,1.833230,-0.089932,0.158527,2
50888,1.772750,-0.039630,0.131827,2
